<font size="6" color="red"> ch3. 연관 분석 </font>

# 1. 연관분석 개요
- 데이터들 사이에서 자주 발생하는 속성을 찾고, 그 속성들 사이에 연관성이 어느 정도 있는지를 분석하는 방법
- 활용분야 : 상품진열, 사기보험적발, 카탈로그 디자인, 신상품 카테로리 구성
```
    - {조건} => {결과}
    {소주, 콜라} => {와인} : 소주와 콜라를 사면 와인을 산다
    {와인} => {소주, 콜라} : 와인을 사면 소주와 콜라를 산다
    이는 서로 다른 의미
    이렇게 도출된 연관성 규칙들은 지지도, 신뢰도, 향상도를 통해 얼마나 의미있는 규칙인지 평가
평가
- 연관분석 관련 지표
    - 지지도(support) = 조건결과항목수 / 전체수
        전체 거래 중 연관성 규칙을 구성하는 항목들이 포함된 거래의 비율
    - 신뢰도(confidence) = 조건결과항목수 / 조건항목수
        조건이 발생했을때 결과가 동시에 일어날 확률
    - 향상도(lift) = 조건결과지지도 / ((조건지지도)*(결과지지도))
        우현이 발생한 규칙인지 아닌지 여부를 보려는 것
        1 : 조건과 결과는 우연한 관계일 뿐 연관성이 전혀 없음
        >1 : 양의 상관관계 (의미있는 연관성을 가진 규칙으로 해석)
        <1 : 음의 상관관계
        
    
ex) 
- {조건} => {결과}   지지도        신뢰도        향상도
- {주스} => {콜라}   2/5(0.4)      2/2(1)        0.4 / (0.4*1) = 1
- {소주} => {맥주}   1/5(0.2)      1/3(0.3)      0.2 / (0.6*0.4) = 0.8333, 0.6=3/5(소주 지지도), 0.4=2/5(맥주 지지도)  
```

In [2]:
import os
os.getcwd()

'C:\\bigdata\\src\\08_ML_DL\\자연어 처리'

In [5]:
%ls cf_basket.csv

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 2878-60C8

 C:\bigdata\src\08_ML_DL\자연어 처리 디렉터리

2021-08-19  오전 09:34               122 cf_basket.csv
               1개 파일                 122 바이트
               0개 디렉터리  347,547,721,728 바이트 남음


In [2]:
# 트랜젝션 데이터 가져오기
import csv
transaction = []
with open('cf_basket.csv', 'r',encoding='utf-8') as f:
    csvdata = csv.reader(f)
    for row in csvdata:
        transaction.append(row)
#     transaction = list(csvdata)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

# 2. 연관성분석
- pip install apyori

## 2.1 연관성 규칙 생성


In [3]:
from apyori import apriori
rules = apriori(transaction, min_support=0.2, min_confidence=0.1)
rules = list(rules)
len(rules)

18

In [4]:
rules

[RelationRecord(items=frozenset({'맥주'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'맥주'}), confidence=0.4, lift=1.0)]),
 RelationRecord(items=frozenset({'소주'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'소주'}), confidence=0.6, lift=1.0)]),
 RelationRecord(items=frozenset({'오렌지주스'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'오렌지주스'}), confidence=0.4, lift=1.0)]),
 RelationRecord(items=frozenset({'와인'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'와인'}), confidence=0.6, lift=1.0)]),
 RelationRecord(items=frozenset({'콜라'}), support=0.8, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'콜라'}), confidence=0.8, lift=1.0)]),
 RelationRecord(items=frozenset({'맥주', '소주'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_ad

In [36]:
print(' 조건 -> 결과    \t 지지도 \t 신뢰도 \t 향상도')
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = [x for x in item[0]]
        lhs = ','.join([x for x in item[0]])
        rhs = [x for x in item[1]]
        rhs = ','.join([x for x in item[1]])
        confidence = item[2]
        lift       = item[3]
        if lift > 1:
            print("{} => {}  \t {} \t {} \t {}".format(lhs, rhs,
                        round(support,2), round(confidence,2), round(lift,2)))

 조건 -> 결과    	 지지도 	 신뢰도 	 향상도
맥주 => 콜라  	 0.4 	 1.0 	 1.25
콜라 => 맥주  	 0.4 	 0.5 	 1.25
소주 => 콜라  	 0.6 	 1.0 	 1.25
콜라 => 소주  	 0.6 	 0.75 	 1.25
콜라 => 맥주,소주  	 0.2 	 0.25 	 1.25
맥주,소주 => 콜라  	 0.2 	 1.0 	 1.25
맥주 => 콜라,와인  	 0.2 	 0.5 	 1.25
콜라 => 맥주,와인  	 0.2 	 0.25 	 1.25
맥주,와인 => 콜라  	 0.2 	 1.0 	 1.25
콜라,와인 => 맥주  	 0.2 	 0.5 	 1.25
소주 => 오렌지주스,콜라  	 0.2 	 0.33 	 1.67
콜라 => 소주,오렌지주스  	 0.2 	 0.25 	 1.25
오렌지주스,소주 => 콜라  	 0.2 	 1.0 	 1.25
오렌지주스,콜라 => 소주  	 0.2 	 1.0 	 1.67
콜라 => 소주,와인  	 0.2 	 0.25 	 1.25
소주,와인 => 콜라  	 0.2 	 1.0 	 1.25


In [37]:
import pandas as pd
result_df = pd.DataFrame(None,
                        columns=['lhs','rhs','support','confidence','lift'])
index = 0
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ','.join([x for x in item[0]])
        rhs = ','.join([x for x in item[1]])
        confidence = item[2]
        lift       = item[3]
        if lift > 1:
            result_df.loc[index] = [lhs, rhs, support, confidence, lift]
            index += 1
result_df.sort_values(by='confidence')

,lhs,rhs,support,confidence,lift
4,콜라,"맥주,소주",0.2,0.250000,1.250000
7,콜라,"맥주,와인",0.2,0.250000,1.250000
11,콜라,"소주,오렌지주스",0.2,0.250000,1.250000
14,콜라,"소주,와인",0.2,0.250000,1.250000
10,소주,"오렌지주스,콜라",0.2,0.333333,1.666667
1,콜라,맥주,0.4,0.500000,1.250000
6,맥주,"콜라,와인",0.2,0.500000,1.250000
9,"콜라,와인",맥주,0.2,0.500000,1.250000
3,콜라,소주,0.6,0.750000,1.250000
0,맥주,콜라,0.4,1.000000,1.250000
